# Transformers pour la classification de texte

L'objectif de ce TP est d'implémenter une version simplifiée d'un Transformer pour résoudre un problème de classification de texte.

Nous utiliserons comme exemple illustratif une base de données présente dans la librairie ```Keras``` consistant en des critiques de films postées sur le site IMDB, accompagnées d'une note qui a été binarisée pour révéler le caracète positif, ou négatif, de la critique.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Implémentation d'un bloc de base de Transformer


<center><img src="https://drive.google.com/uc?id=1w1CyLROPq-EWMd-Spr6wR596QEx1KpNa"> </center>
<caption><center> Figure 1: Schéma de l'architecture de GPT 1</center></caption>

La figure ci-dessus présente l'architecture de GPT-1. Le bloc de base d'un Transformer est composé d'un bloc de *Self-Attention*, d'une couche de ```Layer Normalization``` (similaire à la ```Batch Normalization```), d'une couche dense et enfin d'une nouvelle couche de ```Layer Normalization```.

Pour implémenter la *Self-Attention*, vous pouvez utiliser la fonction ```layers.MultiHeadAttention``` (à vous de regarder quels en sont les paramètres dans la documentation).

**ATTENTION: Pour implémenter un Transformer de type GPT, la couche doit masquer l'attention portée par un token aux tokens suivants !**

**Rappel**: Une couche d'Attention *Multi-Head*  se présente sous la forme ci-dessous à gauche, avec le mécanisme d'attention détaillé à droite :


<center>

<img src="https://drive.google.com/uc?id=1UTozEHtsZ3xy61XJqn_Eug-7mn7bFp9m">
<img src="https://drive.google.com/uc?id=1aTttpp1OOasVVZAi3lWwosh68VnBjQnz">
</center>

D'après vous, combien de paramètres comporte une couche d'attention à 2 têtes, pour un *Embedding* de dimension 32 ?

In [ ]:
class TransformerBlock(layers.Layer):
    # embed_dim désigne la dimension des embeddings maintenus à travers les différentes couches,
    # et num_heads le nombre de têtes de la couche d'attention.
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        # Définition des différentes couches qui composent le bloc
        # Couche d'attention
        self.att =
        # Première couche de Layer Normalization
        self.layernorm1 =
        # Couche Dense (Feed-Forward)
        self.ffn =
        # Deuxième couche de normalisation
        self.layernorm2 =

    def call(self, inputs, training):
        # Application des couches successives aux entrées
        return ...

## Implémentation de la double couche d'Embedding

La séquence d'entrée est convertie en *Embedding* de dimension ```embed_dim```.
L'*Embedding* final est constitué de la somme de deux *Embedding*, le premier encodant un mot, et le second encodant la position du mot dans la séquence.

La couche d'*Embedding* de Keras (```layers.Embedding```) est une sorte de table associant à un indice en entrée un vecteur de dimension ```embed_dim```. Chaque coefficient de cette table est en fait un paramètre apprenable.

D'après vous combien de paramètres contiendrait une couche d'*Embedding* associant un vecteur de dimension 32 à chacun des 20000 mots les plus courants du vocabulaire extrait de la base de données que nous allons utiliser ?
Et combien pour l'*Embedding* qui associe un vecteur de dimension 32 à chaque position d'un séquence de longueur ```maxlen``` ?

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        # Définition des différentes couches qui composent le bloc Embedding
        # Embedding de mot
        self.token_emb =
        # Embedding de position
        self.pos_emb =

    def call(self, x):
        # Calcul de l'embedding à partir de l'entrée x
        # ATTENTION : UTILISER UNIQUEMENT DES FONCTIONS TF POUR CETTE PARTIE
        # Récupération de la longueur de la séquence
        maxlen =
        # Création d'un vecteur [0, 1, ..., maxlen] des positions associées aux
        # mots de la séquence
        positions =
        # Calcul des embeddings de position
        positions_emb = ...
        # Calcul des embeddings de mot
        words_emb = ...
        return

## Préparation de la base de données

In [ ]:
# Taille du vocabulaire considéré (on ne conserve que les 20000 mots les plus courants)
vocab_size = 20000
# Taille maximale de la séquence considérée (on ne conserve que les 200 premiers mots de chaque commentaire)
maxlen = 200

# Chargement des données de la base IMDB
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)

print(len(x_train), "séquences d'apprentissage")
print(len(x_val), "séquences de validation")

# Padding des séquences : ajout de "0" pour compléter les séquences trop courtes
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

## Création du modèle

Pour assembler le modèle final, il faut, partant d'une séquence de longueur ```maxlen```, calculer les Embedding puis les fournir en entrée d'une série de blocs Transformer. Pour ce TP, **commencez par ne mettre qu'un seul bloc Transformer**. Vous pourrez en ajouter plus tard si vous le souhaitez.

Pour construire la tête de projection du réseau, vous pouvez moyenner les activations en sortie du bloc Transformer par élément de la séquence grâce à un *Global Average Pooling* (1D !), à relier à une couche dense (par exemple comportant 20 neurones) et enfin à la couche de sortie du réseau.

In [ ]:
embed_dim = 32  # Dimension de l'embedding pour chaque mot
num_heads = 2  # Nombre de têtes d'attention

# A COMPLETER
inputs = layers.Input(shape=(...))
...
...
outputs = ...

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Enfin vous pouvez lancer l'apprentissage, avec par exemple l'optimiseur Adam. Inutile de lancer de trop nombreuses *epochs*, le réseau sur-apprend très vite !

In [ ]:
# A COMPLETER
model.compile(
    optimizer="adam", loss="...", metrics=["..."]
)
history = model.fit(
    x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val)
)

**Questions subsidiaires**:



1.   Comparez les résultats à ceux d'un LSTM bi-directionnel.
2.   **Plus dur**: GPT bénéficie d'un pré-entraînement non supervisé sur des données issues de gigantesques bases de texte. L'idée, résumée dans l'extrait de l'article copié ci-dessous, consiste à pré-entraîner le modèle à prédire le prochain mot d'une séquence fournie. De larges bases de données, comme [WikiText](https://huggingface.co/datasets/wikitext), permettent de pré-entraîner efficacement le réseau, particulièrement dans notre cas la couche d'*Embedding* qui contient la majorité des paramètres du réseau.

<center><img src="https://drive.google.com/uc?id=1RWPVSAEA5frRvqHkOxw6h1MDRe-fT0sC"> </center>


## Quelques éléments pour aller plus loin

Chargement de la base de données WikiText

In [ ]:
!pip install datasets

from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-103-v1")

print(dataset)

La difficulté est maintenant de travailler cette base de données pour produire des séquences, en réutilisant les mêmes numéros de tokens de la base IMDB utilisée précédemment...